In [3]:
from reader import read_file
import pandas as pd
#from classes import Storage, Generator, LineType, Connection, Node
from typing import List, Callable
import numpy as np
import json
    


file_path = pd.ExcelFile('/Users/nwong/Workspace/School/OpenEMPIRE/Data handler/europe_v50/Generator.xlsx')
file_path2 = pd.ExcelFile('/Users/nwong/Workspace/School/OpenEMPIRE/Data handler/europe_v50/Storage.xlsx')
mic = pd.read_excel(file_path, 'MaxInstalledCapacity', skiprows=2, usecols='A:C')
ipc = pd.read_excel(file_path2, 'EnergyMaxBuiltCapacity', skiprows=2, usecols='A:D')
ipc

,Nodes,StorageTypes,Period,EnergyMaxBuiltCapacity
0,Austria,Hydro Pump Storage,1,500000
1,Bosnia H,Hydro Pump Storage,1,500000
2,Belgium,Hydro Pump Storage,1,500000
3,Bulgaria,Hydro Pump Storage,1,500000
4,Switzerland,Hydro Pump Storage,1,500000
...,...,...,...,...
60,Romania,Li-Ion_BESS,1,500000
61,Serbia,Li-Ion_BESS,1,500000
62,Sweden,Li-Ion_BESS,1,500000
63,Slovenia,Li-Ion_BESS,1,500000


In [19]:
def sum_strat(filtered_data: pd.DataFrame, group_by_cols: List[str], agg_col: str):
    return filtered_data.groupby(group_by_cols).sum(agg_col)

def mean_strat(filtered_data: pd.DataFrame, group_by_cols: str, agg_col: str):
    return filtered_data.groupby(group_by_cols).mean(agg_col)

def sum_countries(filtered_data: pd.DataFrame, group_by_cols: List[str], **kwargs):
    return filtered_data[group_by_cols].sum(axis=1)

def construct_super(data: pd.DataFrame,
                    s_name: str, 
                    child_nodes: List[str], 
                    group_by_cols: List[str], 
                    agg_col: str, 
                    agg_strat: Callable):
    
    f_data = data[data.Nodes.isin(child_nodes)]
    agg_data = agg_strat(f_data, group_by_cols, agg_col)
    agg_data['Nodes'] = s_name
    return agg_data

def combine_nodes(data: pd.DataFrame, 
                  s_nodes: dict, 
                  group_by_cols: List[str], 
                  agg_col:str, 
                  agg_strat: Callable):
    dfs = [construct_super(data=data, 
                           s_name=s_node, 
                           child_nodes=child_nodes, 
                           group_by_cols=group_by_cols,
                           agg_col=agg_col, 
                           agg_strat=agg_strat) for s_node, child_nodes in s_nodes.items()]
    return pd.concat(dfs)

def construct_tab_files(config_path: str):
    with open(config_path) as json_file:
        config = json.load(json_file)
        for file in config:
            file_path = pd.ExcelFile(file['file_path'])
            for sheet in file['agg_sheets']:
                sheet_df = pd.read_excel(file_path, sheet['sheet_name'], skiprows=sheet['skiprows'], usecols=sheet['usecols'])
                df = combine_nodes(data=sheet_df, 
                              s_nodes={'Nordics': ['Norway', 'Sweden', 'Denmark'], 'Europe': ['France', 'Germany']},
                              group_by_cols=sheet['group_by_cols'],
                              agg_col=sheet['agg_col'],
                              agg_strat=sum_strat)
                df.to_csv("nic_test/" + file['file_name'] + "_" + sheet['sheet_name'] + '.tab', sep='\t')
                
#construct_tab_files("agg_config.json")

In [31]:
file_path = pd.read_csv('/Users/nwong/Workspace/School/OpenEMPIRE/Data handler/europe_v50/ScenarioData/electricload.csv')
#mic = pd.read_excel(file_path, 'electricload')
file_path
df = file_path[['time', 'hour', 'dayofweek', 'month']]
combine_nodes(file_path, {'Nordics': ['NO', 'SE', 'DK'], 'Europe': ['DE', 'FR', 'BE']}, )
new = sum_countries(file_path, group_by_cols=['BG', 'CZ', 'DE'])
df['countries'] = new
df

<ipython-input-31-5d6d15d19e76>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['countries'] = new


,time,hour,dayofweek,month,countries
0,01/01/2015 00:00,0,3,1,52179.3375
1,01/01/2015 01:00,1,3,1,50677.7125
2,01/01/2015 02:00,2,3,1,49424.8850
3,01/01/2015 03:00,3,3,1,49010.6650
4,01/01/2015 04:00,4,3,1,49065.3825
...,...,...,...,...,...
43819,31/12/2019 19:00,19,1,12,58508.4050
43820,31/12/2019 20:00,20,1,12,56647.0375
43821,31/12/2019 21:00,21,1,12,56129.5750
43822,31/12/2019 22:00,22,1,12,54201.4425


In [46]:
import itertools
dict = {'Nordics': ['NO', 'SE', 'DK'], 'Europe': ['DE', 'FR', 'BE']}
test = [value for value in dict.values()]
list(itertools.chain.from_iterable(test))

['NO', 'SE', 'DK', 'DE', 'FR', 'BE']